Insert random sequences into the training set

In [3]:
#%%
from array import array
from cmath import nan
import statistics
from tkinter.ttk import Separator
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
import torchvision
import torchvision.transforms as transforms
from itertools import chain
from sklearn import metrics as met
import pickle
import icecream as ic
import torchsummary
import matplotlib.pyplot as plt
import pathlib
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
import util
from torchmetrics import Accuracy
from torch.utils.data.datapipes.datapipe import _IterDataPipeSerializationWrapper, _MapDataPipeSerializationWrapper
from torch.utils.data import default_convert
from tqdm import tqdm
import time



/mnt/storageG1/lwang/miniconda3/envs/ml-g1/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
seqs_df, res_all = util.load_data.get_main_dataset()
N_samples = seqs_df.shape[0]
DRUGS = util.DRUGS
LOCI = seqs_df.columns
assert set(DRUGS) == set(res_all.columns)
N_drugs = len(DRUGS)

# load the CRyPTIC samples as test data
seqs_cryptic, res_cryptic = util.load_data.get_cryptic_dataset()
# make sure the loci are in the same order as in the training data
seqs_cryptic = seqs_cryptic[seqs_df.columns]

spread_sheet = pd.read_csv('tb_data_2022_08_19.csv', header=0)

/mnt/storageG1/lwang/miniconda3/envs/ml-g1/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (12,15,16,17,19,20,47,48,49,51,53,54,56,57,58,59,62,63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
from random import choice
def rand_seq_gen(length):

   DNA=""
   
   for count in range(length):
      DNA+=choice("CGTA")
   return DNA

In [6]:
#generating random sample in the training dataset
from random import seed
from random import randint
from tqdm import tqdm

rand_size = int(23039*0.05)
random_df = pd.DataFrame(columns=[seqs_df.columns])
rand_res_df = pd.DataFrame(0, index=np.arange(rand_size), columns=[res_all.columns])

#get mean length of each loci
loci_len = {}
for x in seqs_df.columns:
    loci_len[x] = seqs_df[x].str.split("").str.len().mean().round()

#create instances with random sequences
seqs_df_rand, res_all_rand = seqs_df, res_all
target_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] #dr to be added in corresponding to rand seq

rand_size = int(len(seqs_df)*0.05) #amount of rand seq

seed(42)
# generate some integers
for _ in tqdm(range(rand_size)):
	insert_pos = randint(0, len(seqs_df))
	list_ = [] #creating rand seq
	for k, v in loci_len.items():
		list_.append(rand_seq_gen(int(v)))

	seqs_df_rand = pd.DataFrame(np.insert(seqs_df_rand.values, insert_pos, list_, axis=0)) 
	res_all_rand = pd.DataFrame(np.insert(res_all_rand.values, insert_pos, target_list, axis=0)) 
	# for x in range(rand_size):
	# 	list_ = []
	# 	for k, v in loci_len.items():
	# 		list_.append(rand_seq_gen(int(v)))
	# 	random_df = random_df.append(pd.DataFrame([list_], columns=[seqs_df.columns]), ignore_index=True)
seqs_df_rand.columns = seqs_df.columns
res_all_rand.columns = res_all.columns


100%|██████████| 1151/1151 [01:20<00:00, 14.24it/s]


In [7]:
from pathlib import Path  
filepath = Path('/mnt/storageG1/lwang/TB-AMR-CNN/Julian/data/rand/seq_5percent_rand.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
seqs_df_rand.to_csv(filepath, index=False, index_label=False)

filepath = Path('/mnt/storageG1/lwang/TB-AMR-CNN/Julian/data/rand/res_5percent_rand.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)
res_all_rand.to_csv(filepath, index=False, index_label=False)